Niniejszy skrypt służy do pobrania danych API ze strony https://www.entsoe.eu/
Strona internetowa ENTSO-E (Europejskiej Sieci Operatorów Elektroenergetycznych Systemów Przesyłowych - European Network of Transmission System Operators for Electricity) zawiera szeroki zakres informacji związanych z europejskim rynkiem energii elektrycznej, bezpieczeństwem systemu i rozwojem sieci.

Skryptu nie należy uruchamiać ponownie - pobranie danych odbywa się jednorazowo.

In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from entsoe import EntsoePandasClient
import os
from dotenv import load_dotenv
import time

print("--- Start Skryptu 1 (Zakres 2015-2025): Pobieranie Danych Surowych ---")

# --- 1. ŁADOWANIE KLUCZA API ---
print("Ładowanie klucza API z pliku .env...")
load_dotenv()
API_KEY = os.getenv("ENTSOE_API_KEY")

if not API_KEY:
    print("BŁĄD: Nie znaleziono klucza 'ENTSOE_API_KEY'.")
    print("Upewnij się, że masz plik .env z kluczem.")
else:
    print(f"Klucz API wczytany (zaczyna się od: '{API_KEY[:4]}...')")

# --- 2. KONFIGURACJA ---
if API_KEY:
    client = EntsoePandasClient(api_key=API_KEY)

    # Ustawiamy 10+ lat danych
    start_date = pd.Timestamp('2015-01-01', tz='UTC')
    
        # Pobieramy dane do 1 listopada 2025 (czyli do końca października 2025)
    end_date = pd.Timestamp('2025-11-01', tz='UTC')
    # --- ---

    # Kody krajów i stref
    country_code_de = 'DE'
    
    # Definiujemy obie strefy cenowe i datę podziału. Zakres ustalony na podstawie wytycznych SMARD. Niemcy w latach 2015-2018 miały wspólną strefę cenową z Austrią i Luksemburgiem, a od 2018-10-01 tylko z Luksemburgiem.
    country_code_prices_old = 'DE_AT_LU' # Niemcy/Austria/Luksemburg
    country_code_prices_new = 'DE_LU'    # Niemcy/Luksemburg
    split_date = pd.Timestamp('2018-10-01', tz='UTC')

    print(f"\nUstawiony okres: {start_date} do {end_date}")

    # --- 3. POBIERANIE DANYCH (Zadania Proste) ---
    simple_tasks = {
        'raw_generation_de.csv': (client.query_generation, country_code_de),
        'raw_load_de.csv': (client.query_load, country_code_de),
        'raw_load_forecast_de.csv': (client.query_load_forecast, country_code_de),
    }

    print("\nRozpoczynam pobieranie 3 zestawów danych (Generacja, Zużycie, Prognoza)...")
    
    for file_name, (query_function, country_code) in simple_tasks.items():
        print(f"  Pobieranie: {file_name}...")
        try:
            data = query_function(country_code, start=start_date, end=end_date)
            data.to_csv(file_name)
            print(f"  ✅ SUKCES: Zapisano '{file_name}' ({len(data)} rekordów).")
            time.sleep(1) # Przerwa dla API
        except Exception as e:
            print(f"  ❌ BŁĄD: Nie udało się pobrać {file_name}. Błąd: {e}")

    # --- 4. POBIERANIE DANYCH (Zadanie Złożone - Ceny) ---
    print("\nRozpoczynam pobieranie danych cenowych (w 2 częściach)...")
    
    try:
        # CZĘŚĆ 1: Dane historyczne (do 30.09.2018)
        print(f"  Pobieranie cen (Stare): {country_code_prices_old} (do {split_date})...")
        prices_old = client.query_day_ahead_prices(
            country_code_prices_old,
            start=start_date,
            end=split_date
        )
        print(f"  Pobrano {len(prices_old)} rekordów (stare).")
        time.sleep(1)

        # CZĘŚĆ 2: Dane nowe (od 01.10.2018)
        print(f"  Pobieranie cen (Nowe): {country_code_prices_new} (od {split_date})...")
        prices_new = client.query_day_ahead_prices(
            country_code_prices_new,
            start=split_date,
            end=end_date
        )
        print(f"  Pobrano {len(prices_new)} rekordów (nowe).")

        # Łączenie i zapisywanie
        combined_prices = pd.concat([prices_old, prices_new])
        
        if not combined_prices.index.is_monotonic_increasing:
             print("  OSTRZEŻENIE: Indeks cen nie jest monotoniczny. Sortowanie...")
             combined_prices = combined_prices.sort_index()

        output_file_prices = 'raw_prices_de_combined.csv'
        combined_prices.to_csv(output_file_prices)
        print(f"SUKCES: Zapisano połączone ceny w '{output_file_prices}' ({len(combined_prices)} rekordów).")
        
    except Exception as e:
        print(f"BŁĄD: Nie udało się pobrać danych cenowych. Błąd: {e}")

    print("\n--- Zakończono Skrypt 1 ---")

else:
    print("\nSkrypt zatrzymany z powodu braku klucza API.")

--- Start Skryptu 1 (Zakres 2015-2025): Pobieranie Danych Surowych ---
Ładowanie klucza API z pliku .env...
Klucz API wczytany (zaczyna się od: '3869...')

Ustawiony okres: 2015-01-01 00:00:00+00:00 do 2025-11-01 00:00:00+00:00

Rozpoczynam pobieranie 3 zestawów danych (Generacja, Zużycie, Prognoza)...
  Pobieranie: raw_generation_de.csv...
  ✅ SUKCES: Zapisano 'raw_generation_de.csv' (379862 rekordów).
  Pobieranie: raw_load_de.csv...
  ✅ SUKCES: Zapisano 'raw_load_de.csv' (379862 rekordów).
  Pobieranie: raw_load_forecast_de.csv...
  ✅ SUKCES: Zapisano 'raw_load_forecast_de.csv' (379862 rekordów).

Rozpoczynam pobieranie danych cenowych (w 2 częściach)...
  Pobieranie cen (Stare): DE_AT_LU (do 2018-10-01 00:00:00+00:00)...
  Pobrano 32759 rekordów (stare).
  Pobieranie cen (Nowe): DE_LU (od 2018-10-01 00:00:00+00:00)...
  Pobrano 64351 rekordów (nowe).
  ✅ SUKCES: Zapisano połączone ceny w 'raw_prices_de_combined.csv' (97110 rekordów).

--- Zakończono Skrypt 1 ---
